# Qiskit Fall Fest @ Yonsei 2025 : Challenger Hackathon

> Qiskit Fall Fest @ Yonsei 2025 챌린저 해커톤에 오신 모든 분들을 환영합니다! 🎉  
> 이 노트북은 양자 컴퓨팅과 Qiskit에 익숙한 여러분을 위해 도전적이고 새로운 경험을 주기 위해 제작되었습니다.
> 끝까지 완료하시면 챌린저 챌린지 경품 추첨에 자동으로 응모되니, 도전과 함께 행운의 기회도 잡아보세요!

**진행 방법:**

1.  **개념 학습**: 각 주제에 대한 설명을 꼼꼼히 읽어보세요.
2.  **코드 작성**: 설명 아래 셀에서 `TODO`라 써있는 부분에 문제에 맞는 코드를 작성합니다.
3.  **정답 확인**: 코드 작성이 끝나면, 바로 아래 `Grader Cell`을 실행해 정답 여부를 확인하세요.
4.  **경품 응모**: 마지막 문제까지 모두 통과하면 경품 추첨에 자동으로 응모됩니다.

### GOOD LUCK 🤞

## Setup

먼저 아래 셀을 실행해서 이번 챌린지에서 필요한 패키지들을 설치하고 불러옵니다

In [ ]:
! pip install numpy scipy matplotlib qiskit qiskit-aer qiskit-ibm-runtime qiskit-addon-obp qiskit-addon-utils
! pip install requests


import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import SamplerV2 as Sampler, EstimatorV2 as Estimator, QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit import Parameter
from scipy.optimize import minimize

from qiskit.primitives import StatevectorEstimator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import CouplingMap
from qiskit.synthesis import LieTrotter

from qiskit_addon_obp import backpropagate
from qiskit_addon_obp.utils.simplify import OperatorBudget
from qiskit_addon_obp.utils.truncating import setup_budget

from qiskit_addon_utils.problem_generators import (
    generate_xyz_hamiltonian,
    generate_time_evolution_circuit,
)
from qiskit_addon_utils.slicing import slice_by_gate_types, combine_slices


from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import (
    Session,
    EstimatorV2 as Estimator,
    SamplerV2 as Sampler,
    EstimatorOptions,
)
from qiskit_aer import AerSimulator


import qff25_yonsei_challenger_grader as grader

In [ ]:
# Initialize grader

user_name = # 동명이인이 있을 수 있으니, (이름_전화번호 4자리)를 입력하세요. ex)홍길동_3456
grader.initialize_grader(user_name)

In [ ]:
# 계정이 잘 저장되었는지 확인합니다
service = QiskitRuntimeService()
service.saved_accounts()

---
## Part 1. 연산자 역전파(Operator Backpropagation)

**연산자 역전파(Operator Backpropagation, OBP)** 는 양자 회로의 뒷 부분에 있는 연산들을 측정하려는 observable에 흡수시키는 기술입니다. 이 과정은 일반적으로 관측량의 term을 늘리는 대가로 양자 하드웨어에서 실행되는 회로의 깊이를 줄여줍니다.

이 과정은 일반적으로 회로 깊이가 줄어드는 이득을 얻는 대신, 관측량의 항의 term 개수가 늘어나는 trade-off가 있습니다. 그래서 Truncation과 같은 기법이 함께 사용됩니다.

* #### Background
`Estimator` primitive를 통해 구하려는 expectation value는 초기 상태 $|\psi\rangle$와 전체 회로 $U$에 대해 $\langle O\rangle_{U|\psi\rangle} = \langle\psi|U^\dagger OU|\psi\rangle$로 표현됩니다.

전체 회로 $U$를 양자 컴퓨터에서 실행할 앞부분($U_Q$)과 고전적으로 계산할 뒷부분($U_C$)으로 나눕니다. ($U=U_C U_Q$) 이를 위 기댓값 식에 대입하면 다음과 같습니다. $\langle O \rangle_{U|\psi\rangle} = \langle\psi|(U_CU_Q)^\dagger O (U_C U_Q)|\psi\rangle = \langle\psi|U_Q^\dagger(U_C^\dagger O U_C)U_Q|\psi\rangle$

여기서 괄호 안의 $U_C^\dagger O U_C$를 새로운 관측량 $O'$으로 정의하면 $\langle \psi|U_Q^\dagger O' U_Q |\psi\rangle$이 됩니다. 

양자회로 $U_C$가 양자 '상태'에 미치는 영향을 고전 컴퓨터로 시뮬레이션하는 것은 어렵습니다. 하지만 $U_C^\dagger O U_C$ 연산은 '상태'가 아닌 '연산자' 자체를 다루기 때문에 고전 컴퓨터로 효율적으로 계산이 가능합니다.

($U_C^\dagger O U_C$ 연산은 거대한 행렬을 직접 곱하는 대신, $O$를 구성하는 각 pauli 항들이 $U_C$의 게이트를 지날 때 어떻게 변하는지를 pauli 대수학 규칙에 따라 추적해서 효율적으로 계산할 수 있습니다.)

결과적으로, 원래의 문제였던 $\langle O\rangle_{U|\psi\rangle}$계산에서 circuit depth가 얕아진 $U_Q$에 대해 좀 더 복잡한 관측량 $O'$의 기댓값을 측정하는 $\langle O' \rangle_{U_Q|\psi\rangle}$을 구하는 문제로 바뀌게 됩니다.

* #### Truncation

OBP를 통해 $U_Q$의 깊이를 줄여 이득을 얻는 대신, 측정해야 할 새로운 observable $O'$의 항이 늘어나 고전적인 후처리 계산량과 총 측정 횟수가 늘어나는 trade-off가 있습니다.

이 trade-off를 balance하기 위해 $O'$의 항들 중에서 크기가 매우 작은 계수를 가진 항을 버리는 **Truncation** 을 사용할 수 있습니다. Truncation을 통해 측정 횟수를 줄일 수 있지만, 그 대가로 결과에 오차가 생깁니다.

따라서 truncation 레벨을 적절히 조절하여, 양자 회로의 깊이 감소에 따른 노이즈 감소 효과와 truncation에 의한 오차 사이에서 균형점을 찾는 것이 중요합니다.

* #### Clifford Perturbation Theory 

`qiskit-addon-obp`에는 Clifford perturbation theory을 기반으로 OBP가 구현돼있습니다.

양자 게이트는 고전적으로 시뮬레이션 하기 쉬운 Clifford gate와 어려운 Non-Clifford gate로 나눌 수 있습니다.
* Clifford gate (ex. X, H, S, CNOT) : 연산이 비교적 단순하여, 이들은 backpropagate하는 계산 비용이 낮습니다. 따라서 observable의 복잡도를 거의 증가시키지 않습니다.

* Non-Clifford gate (ex. T, rotational gates) : 이 게이트들은 역전파하면 observable의 항개수가 크게 늘어가 고전 컴퓨터의 계산 비용이 커집니다.

이 방식은 역전파할 때 발생하는 오버헤드가 $U_C$의 non-Cliffordness($U_C$가 얼마나 non-Clifford들로 이루어져있는지)에 비례한다는 장점이 있습니다. 즉, $U_C$가 Clifford 게이트로 이루어져 있다면 OBP를 더 빠르고 효율적이게 수행할 수 있습니다.

이 방법은 $U_C$를 $U_C = \prod_{s=1}^S u_s=u_s...u_2u_1$로 슬라이스 하고, 각 슬라이스를 하나씩 역전파 하면서, 새로운 observable $O'$가 얼마나 복잡해지는지 계속 추적합니다.

만약 $O'$가 너무 복잡해지거나, 역전파로 인한 오차가 설정된 미리 설정해둔 error budget을 초과할 것 같으면, truncation을 합니다.

---
`qiskit-addon-obp`패키지의 핵심은 `backpropagate()`함수입니다. 이 함수는 긴 양자 회로의 일부를 잘라내어 고전적인 계산으로 대체하고, 그 결과로 더 짧아진 회로와 더 복잡해진 observable을 만들어 냅니다.

`backpropagate()`함수의 사용법은 크게 다음과 같습니다.

##### 1. 문제 정의

먼저, 시뮬레이션할 물리 시스템과 observable을 정의합니다.

* Hamiltonian : 시뮬레이션할 시스템의 해밀토니안을 정의합니다.

* Observable : 측정할 observable을 정의합니다.
위 예시에 대해서는 $\langle Z_0 \rangle$으로 할 수 있습니다.

* 전체 회로($U$) : 해밀토니안에 따라 시스템이 특정 시간 동안 어떻게 변하는지 나타내는 전체 양자 회로입니다.


##### 2. 회로 슬라이스 및 error budget 정의하기

다음으로, OBP를 적용하기 위한 두 가지 중요한 요소를 결정해야 합니다.

* Circuit Slice : 전체 회로 $U$에서 역전파할 뒷부분 $U_C$를 작은 회로 조각(슬라이스)들의 리스트 형태로 잘라냅니다.

* Error Budget : 역전파 과정에서 관측량이 지나치게 복잡해지는 것을 제어하기 위한 규칙을 정합니다.
    * `TruncationErrorBudget` : 허용할 이론적 오차의 최대치
    * `OperatorBudget` : 최종 observable이 가질 수 있는 항의 최대 개수

##### 3. `backpropagate()`함수 실행

이제 위에서 준비한 인자들을 `backpropagate()`함수에 전달하여 실행합니다.

* Input : 원래 Observable($O$), $U_C$, Error Budget

* Output : 새로운 Observable($O'$), $U_Q$


##### 4. 기댓값 계산

마지막으로 `Estimator` primitve를 사용해 3단계에서 얻은 결과물의 기댓값을 계산합니다. 

`Estimator.run(circuits=U_Q, observables=O')`

---
이제 위 단계대로 `qiskit-addon-obp`패키지를 사용해봅시다.

Heisenberg XYZ chain의 time evolution 시뮬레이션을 OBP를 활용해 효율적으로 해봅시다.

### **문제 1-1 :**

다음 기능을 수행하는 코드를 작성하세요 :

**10**개 큐비트의 선형 체인(linear chain)을 `CouplingMap`을 사용하여 형성합니다.

In [ ]:
num_qubits = # TODO

layout = # TODO
coupling_map = CouplingMap(layout)

In [ ]:
# Grader Cell
grader.grade_1_1(coupling_map)

---
다음으로, `coupling_map`을 기반으로 하이젠베르크 XYZ 해밀토니안을 모델링하는 파울리 연산자(Pauli operator)를 생성합니다.

### **문제 1-2**

$${\hat{\mathcal{H}}_{XYZ} = \sum_{(j,k)\in E} (J_{x} \sigma_j^{x} \sigma_{k}^{x} + J_{y} \sigma_j^{y} \sigma_{k}^{y} + J_{z} \sigma_j^{z} \sigma_{k}^{z}) + \sum_{j\in V} (h_{x} \sigma_j^{x} + h_{y} \sigma_j^{y} + h_{z} \sigma_j^{z})}$$

여기서 상호작용 항의 계수는 $${J = (J_x, J_y, J_z) = (\frac{\pi}{8}, \frac{\pi}{4}, \frac{\pi}{2})}$$이고, 외부 자기장 항의 계수는$${h = (h_x, h_y, h_z) = (\frac{\pi}{3}, \frac{\pi}{6}, \frac{\pi}{9})}$$ 으로 합니다.

이렇게 생성된 해밀토니안으로부터, **시간($t=0.2$)** 동안의 동역학을 시뮬레이션하는 양자 회로를 만드세요.

Time evolution은 2개의 트로터 스텝(`reps=2`)을 사용하는 리에-트로터 분해(Lie-Trotter decomposition) 방식으로 근사하게 하세요.

In [ ]:
hamiltonian = generate_xyz_hamiltonian(
    coupling_map,
    coupling_constants= # TODO ,
    ext_magnetic_field= # TODO ,
)
print(hamiltonian)

circuit = generate_time_evolution_circuit(
    hamiltonian,
    time= # TODO ,
    synthesis= #TODO ,
)

circuit.draw("mpl", style="iqp", scale=0.6)

In [ ]:
# Grader Cell
grader.grade_1_2(hamiltonian, circuit)

---

### **문제 1-3**
Observable을 $O = \frac{1}{N} \sum X_i $로 지정합니다.

`SparsePauliOp`를 이용해 `observable`을 만드세요.


In [ ]:
observable = SparsePauliOp.from_sparse_list(
    # TODO
)
print(observable)

In [ ]:
# Grader Cell

grader.grade_1_3(observable)

---

이제 `backpropagate()`의 입력 데이터인 회로 슬라이스와 error budget을 준비하고 실행합시다.

### **문제 1-4 :**

다음 기능을 수행하는 코드를 작성하세요 :

1. `circuit`을 `slice_by_gate_types()`를 이용해 슬라이스 하세요.

* 참고 : 회로를 슬라이싱하는 방법에 따라 역전파의 성능이 크게 달라집니다. `slice_by_gate_types`은 비슷한 종류의 게이트끼리 묶어서 슬라이스 하는 방법입니다. (ex. RZ 게이트 끼리, CX 게이트 끼리 등등..)

2. `OperatorBudget()`을 사용하여, 역전파된 관측량의 항(QWC 그룹) 개수가 최대 8개를 넘지 않도록 `op_budget` 변수를 만드세요.   

3. `setup_budget()`을 이용해 각 슬라이스를 역전파할 때 허용되는 이론적 오차를 0.05로 설정하고, `trunc_budget` 변수를 만드세요.

4. `combine_slices`를 사용해 새로운 observable에 역전파하지 못한 남은 슬라이스들을 다시 합쳐서 $U_Q$를 만드세요.

In [ ]:
slices = # TODO
print(f"Separated the circuit into {len(slices)} slices.\n")

op_budget = # TODO
trunc_budget = # TODO

bp_observable, remaining_slices, metadata = backpropagate(
    observable,
    slices,
    operator_budget=op_budget,
    truncation_error_budget=trunc_budget,
)
bp_circuit = #TODO

print(f"Backpropagated {metadata.num_backpropagated_slices} slices.")
print(f"The original observable had {len(observable.paulis)} terms.")
print(
    f"New observable has {len(bp_observable.paulis)} terms, which can be combined into {len(bp_observable.group_commuting(qubit_wise=True))} groups."
)
print(
    f"After truncation, the error in our observable is bounded by {metadata.accumulated_error(0):.3e}"
)
print("The remaining circuit after backpropagation(U_Q) looks as follows:")
bp_circuit.draw("mpl", scale=0.6)

In [ ]:
# Grader Cell
grader.grade_1_4(bp_observable, bp_circuit, observable, circuit)

#### 문제 1의 결과와 문제 2의 결과 회로를 비교해보면, 실제로 양자컴퓨터에서 돌려야하는 회로($U_Q$)가 훨씬 짧아졌음을 확일할 수 있습니다.
---

#### 이제 OBP를 한 회로와 observable로 기댓값을 계산해보고, 실제 기댓값과 비교해봅시다.

`StatevectorEstimator`로 실제 기댓값을 계산합니다.

In [ ]:
ideal_estimator = StatevectorEstimator()

result_exact = ideal_estimator.run([(circuit, observable)]).result()[0].data.evs.item()
print(f"Exact expectation value: {result_exact}")

Noisy simulator에서의 기댓값을 확인합니다.

In [ ]:
service = QiskitRuntimeService(name="YOUR_INSTANCE_NAME")
backend = service.least_busy()
backend_sim = AerSimulator.from_backend(backend)

In [ ]:
pm = generate_preset_pass_manager(backend=backend_sim, optimization_level=3)

circuit_isa = pm.run(circuit)
observable_isa = observable.apply_layout(circuit_isa.layout)

bp_circuit_isa = pm.run(bp_circuit)
bp_obs_isa = bp_observable.apply_layout(bp_circuit_isa.layout)

In [ ]:
options = EstimatorOptions()
options.default_precision = 0.011
options.resilience_level = 2

estimator = Estimator(mode=backend_sim, options=options)

pub = (circuit_isa, observable_isa)
bp_pub = (bp_circuit_isa, bp_obs_isa)

job = estimator.run([pub, bp_pub])

result_no_bp = job.result()[0].data.evs.item()
result_bp = job.result()[1].data.evs.item()

std_no_bp = job.result()[0].data.stds.item()
std_bp = job.result()[1].data.stds.item()


print(f"Expectation value without backpropagation: {result_no_bp} ± {std_no_bp}")
print(f"Backpropagated expectation value: {result_bp} ± {std_bp}")

실제값과 obp 적용 전, 후, 절단(trncation) 적용에 따른 결과를 비교하여 확인합니다.

In [ ]:
methods = [
    "No backpropagation",
    "Backpropagation",
]
values = [result_no_bp, result_bp]
stds = [std_no_bp, std_bp]

ax = plt.gca()
plt.bar(methods, values, color="#a56eff", width=0.4, edgecolor="#8a3ffc")
plt.axhline(result_exact)
ax.set_ylim([0.20, 0.30])
plt.text(0.2, 0.285, "Exact result")
ax.set_ylabel(r"$M_X$", fontsize=12)

---
## Optimal OBP Budget

OBP를 사용할 때 OperatorBudget은 결과의 정확도와 계산 비용 사이의 균형을 맞추는 중요한 파라미터입니다. 하지만 역전파 과정 중 관측량의 복잡도(QWC 그룹 수)가 어떻게 증가하는지는 미리 알기 어렵습니다.

이번 문제에서는 `backpropagate()` 함수를 실행하여 얻은 `metadata` 정보를 분석함으로써, 주어진 최대 한도(`max_n`) 내에서 실제로 발생 가능한 모든 QWC 그룹 수를 알아내고, 이에 해당하는 `OperatorBudget` 객체들을 생성하는 함수를 완성합니다. 

이는 나중에 다양한 예산 설정에 따른 OBP 효과를 효율적으로 비교하는 데 사용됩니다.

### **문제 1-5 :**

다음 기능을 수행하는 `all_op_budget(slices, max_n)` 함수를 완성하세요.

1. 최대 예산으로 OBP 실행: `backpropagate(max_qwc_groups=max_n)`으로 `metadata`를 얻습니다.

2. QWC 그룹 얻기: `metadata.backpropagation_history`를 순회하며 각 역전파 단계에서 기록된 `num_qwc_groups`를 추출해 `max_qwc_groups_list`에 저장합니다.

3. 결과 정리: 
* `max_qwc_groups_list`에서 중복된 값을 제거하고 오름차순으로 정렬합니다.

* 정렬된 리스트의 각 QWC 그룹 수(`i`)에 대해 `OperatorBudget(max_qwc_groups=i)` 객체를 생성하여 `op_budgets` 리스트를 만듭니다.

* `op_budgets` 리스트와 `max_qwc_groups_list`를 반환합니다.

In [ ]:
# def all_op_budget(slices: list, max_n: int):
#     bp_observable, remaining_slices, metadata = backpropagate(
#         observable, 
#         slices, 
#         operator_budget=OperatorBudget(max_qwc_groups=max_n), 
#     )

#     max_qwc_groups_list = [i.num_qwc_groups for i in metadata.backpropagation_history[:-1]]
#     max_qwc_groups_list = list(set(max_qwc_groups_list))

#     max_qwc_groups_list.sort()
#     print(max_qwc_groups_list)
#     op_budgets = [OperatorBudget(max_qwc_groups=i) for i in max_qwc_groups_list]

#     return op_budgets, max_qwc_groups_list

def all_op_budget(slices: list, max_n: int):
    bp_observable, remaining_slices, metadata = backpropagate(
        observable, 
        slices, 
        operator_budget=OperatorBudget(max_qwc_groups=max_n), 
    )

    max_qwc_groups_list = # TODO
    op_budgets = # TODO

    return op_budgets, max_qwc_groups_list

In [ ]:
slices = slice_by_gate_types(circuit)
print(type(slices))
print(f"Separated the circuit into {len(slices)} slices.")

optimal_op_budgets, qwc_list = all_op_budget(slices, max_n=50)

print(f"Optimial Budgets: {optimal_op_budgets}")
print(f"QWC Groups: {qwc_list}")

이를 통해 회로의 slices와 max_n을 가지고 estimator 실행을 위한 함수를 다음과 같이 정의합니다.

In [ ]:
def make_pubs(slices, observable, pm, max_n):
    op_budgets, _ = all_op_budget(slices, max_n)
    pubs = []
    for op_budget in op_budgets:
        bp_observable, remaining_slices, metadata = backpropagate(
            observable,
            slices,
            operator_budget=op_budget,
        )
        bp_circuit = combine_slices(remaining_slices)

        bp_circuit_isa = pm.run(bp_circuit)
        bp_observable_isa = bp_observable.apply_layout(bp_circuit_isa.layout)
        pubs.append((bp_circuit_isa, bp_observable_isa))

    return pubs


# 시간이 오래 걸릴 수 있습니다.
pubs = make_pubs(slices, observable, pm, max_n=50)
job = estimator.run(pubs)

In [ ]:
# 시간이 오래 걸릴 수 있습니다. (약 10분)
results = [i.data.evs.item() for i in job.result()]
print(results)

이후 결과를 그래프로 그려 확인해봅니다.

In [ ]:
methods = [f"{i}" for i in qwc_list]

ax = plt.gca()
plt.bar(methods, results, color="#a56eff", width=0.4, edgecolor="#8a3ffc")
plt.axhline(result_exact)
ax.set_ylim([0.20, 0.30])
plt.text(0.2, 0.285, "Exact result")
ax.set_xlabel("Max qwc groups", fontsize=12)
ax.set_ylabel(r"$M_X$", fontsize=12)

---
<br><br><br>

---
## Part 2. Bomb Detector

아주 민감해서 빛 한 줄기만 닿아도 터지는 폭탄들이 있다고 상상해 보세요. 이 중 어떤 것은 진짜 작동하는 폭탄이고, 어떤 것은 불량품입니다. 

당신의 임무는 이 폭탄들을 터뜨리지 않고 진짜 작동하는 폭탄만 골라내는 것입니다.

고전적인 방법으로는 불가능해 보입니다. 확인하려면 건드려야 하고, 건드리면 터지니까요! 

하지만 양자역학 원리를 이용하면 이 역설적인 문제를 풀 수 있습니다. 이것이 바로 Elitzur-Vaidman 폭탄 탐지기입니다.

(더 자세한 설명은 이 유튜브 영상을 참고하세요: https://www.youtube.com/watch?v=DNKp4QWc0Ys)

<img src="img/MZ.png" width="40%">

양자 입자(광자)는 파동처럼 행동하며, 중첩 상태가 되면 마치 여러 경로로 동시에 이동하는 것처럼 행동할 수 있습니다.

이 실험에서는 Mach-Zehnder Interferometer라는 장치를 사용합니다. 광자를 두 경로로 나누었다가 다시 합쳐 간섭 효과를 봅니다.

1. 첫 번째 반거울(Beam Splitter)이 광자를 두 개의 다른 경로로 동시에 이동하는 중첩 상태를 만듭니다.
2. 두 경로 중 하나에 우리가 확인할 폭탄을 놓습니다.(설명의 편의를 위해 아래 경로로 하겠습니다.)
3. 두 번째 반거울이 두 경로를 다시 합쳐 간섭을 일으킵니다.


#### - Case 1) 폭탄이 없을 때 (폭탄이 불량일 때)

1번 검출기 : 아래 경로, 윗 경로 파동이 서로를 정확히 상쇄시키는 상쇄 간섭이 일어나 빛이 도달하지 않아 1번 검출기에서 광자가 검출되지 않습니다.

2번 검출기 : 아래 경로, 윗 경로 파동이 서로를 완벽하게 증폭시키는 보강 간섭이 일어나서 빛이 도달해 2번 검출기에서 광자가 검출됩니다.

#### - Case 2) 폭탄이 있을 때

폭탄 터짐 : 아무 검출기도 울리지 않습니다. 실제로 폭탄이 터졌습니다.

2번 검출기 울림 : 폭탄이 있었는데도 2번 검출기가 울렸으면, 운 좋게 광자가 위쪽 경로(안전한 경로)로 가서 폭발은 피했고, 간섭이 깨진 상태에서 우연히 2번 검출기가 감지되었을 수 있습니다.

1번 검출기 울림 : 폭탄이 없는 경우였으면, 1번 검출기 방향은 상쇄간섭돼서 아무것도 감지 되지 않아야 합니다.

하지만, 1번에서 검출 됐다면, 무언가에 의해서 상쇄간섭이 파괴되었다는 뜻인거고, 그 간섭을 파괴한것이 폭탄이라는 뜻입니다. --> 폭탄 터트리지 않고 폭탄을 감지했습니다. 😃

#### 따라서 이 실험은 세가지 결과가 가능합니다.

1. 폭탄 터짐 : 광자가 폭탄이 있는 경로를 '선택'하여 폭탄과 직접 상호작용하고 터집니다. (실패)

2. 폭탄 감지 : 자가 폭탄이 없는 안전한 경로를 '선택'했지만, 중첩이 깨진 여파로 인해 이전에는 가지 않았던 1번 검출기로 가게 됩니다. 이 결과는 폭탄을 터뜨리지 않고도 그 존재를 100% 확신시켜 줍니다. (성공)

3. 알 수 없음 : 폭탄이 실제로 없어서 2번 감지기가 울렸을 수 도 있고, 폭탄이 있지만 광자가 안전한 경로를 선택해 2번 감지기가 울렸을 수 도 있습니다. 이 시도로는 아무 정보도 얻지 못했습니다.
---
#### 먼저 폭탄이 없는 상황에서의 Mach-Zehnder 간섭계를 양자회로로 구현해보겠습니다.

* Beam Splitter : 광자를 두 경로로 50% 확률로 나누거나 합치는 장치입니다. 이는 양자 회로에서 Hadamard 게이트로 구현할 수 있습니다.

* 큐비트: 두 개의 큐비트를 사용합니다.

    1. q0: 광자의 경로 정보를 나타냅니다.

    2. q1: 두 경로를 구분하거나 얽힘(entanglement)을 만드는 데 사용됩니다.

* 클래시컬 비트: 최종 결과를 저장하기 위해 하나가 필요합니다.

### **문제 2-1 :**

다음 기능을 수행하는 make_no_bomb_circuit() 함수를 만드세요.

1. 2개의 큐비트와 1개의 클래시컬 비트를 가지는 `QuantumCircuit`을 만드세요.

2. 첫 번째 빔 스플리터:

    - q0에 아다마르 게이트를 적용하여 중첩 상태를 만듭니다.

    - q0를 제어 큐비트로, q1을 목표 큐비트로 하는 CNOT(CX) 게이트를 적용합니다.

    - 가독성을 위해 `qc.barrier()`를 추가합니다.

3. 두 번째 빔 스플리터:

    - q0를 제어 큐비트로, q1을 목표 큐비트로 하는 CNOT(CX) 게이트를 적용합니다.

    - q0에 아다마르 게이트를 적용하여 경로를 다시 합치고 간섭을 일으킵니다.

    - 가독성을 위해 `qc.barrier()`를 추가합니다.

4. q0 큐비트를 측정하여 그 결과를 클래시컬 비트 0번에 저장하세요.

In [ ]:
def make_no_bomb_circuit():
    # TODO

In [ ]:
# Grader Cell
grader.grade_2_1(make_no_bomb_circuit())

---

### **문제 2-2 :**

`AerSimulator`를 백엔드로 해서 `Sampler`로 `no_bomb_circuit`을 `shots=1024`로 실행해 히스토그램을 얻습니다.

In [ ]:
# TODO

no_bomb_circuit = # TODO
# TODO
counts_no_bomb = # TODO

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
no_bomb_circuit.draw("mpl", ax=axes[0])
plot_histogram(counts_no_bomb, ax=axes[1])
plt.show()

In [ ]:
# Grader Cell
grader.grade_2_2(counts_no_bomb)

#### 폭탄이 없어서 전부 다 Detector 2에서 검출됐습니다. (측정결과가 0이면 2번 검출기가, 1이면 1번 검출기가 울린겁니다.)
---
#### 이번에는 Mach-Zehnder 간섭계 경로 중간에 작동하는 폭탄이 놓여 있는 상황을 양자회로로  구현해보겠습니다.

**핵심 아이디어: 폭탄 = 측정**

Elitzur-Vaidman 폭탐 탐지기에서 폭탄은 광자가 자신의 경로로 지나가는지 **확인**합니다. 양자역학에서 이 **확인** 행위가 **측정**에 해당합니다.

따라서, 회로 중간에 폭탄을 놓는 것은 해당 경로를 지나가는 큐비트를 중간에 `measure()`하는 것으로 완벽하게 모델링할 수 있습니다. 이 `measure()` 연산이 바로 폭탄의 존재와 상호작용을 시뮬레이션하는 핵심입니다.

### **문제 2-3 :**

다음 기능을 수행하는 make_bomb_circuit() 함수를 만드세요.

1. 2개의 큐비트와 2개의 클래시컬 비트를 가지는 `QuantumCircuit`을 만드세요. (`c0`은 최종 결과, `c1`은 폭발 여부를 위한 레지스터입니다.)

2. 첫 번째 빔 스플리터:

    - 이전 문제(`make_no_bomb_circuit()`)와 동일하게, H 게이트와 CX 게이트를 사용하여 `q0`를 중첩시키고 `q1`과 얽습니다. 
    - `qc.barrier()`를 추가하고 `label='BOMB'`을 추가해 폭탄이 올 자리라고 표시합니다.

3. 폭탄:

    - `q1`을 측정하여 그 결과를 `c1`에 저장하세요. 이것이 폭탄의 존재를 모델링합니다.

    - 가독성을 위해 qc.barrier()를 추가합니다.

4. 두 번째 빔 스플리터: 이전 문제와 동일하게, CX 게이트와 H 게이트를 사용하여 경로를 다시 합칩니다. `qc.barrier()`를 추가합니다.

5. 최종 측정: `q0` 큐비트를 측정하여 그 결과를 `c0`에 저장하세요.

In [ ]:
def make_bomb_circuit():
    # TODO

In [ ]:
# Grader Cell
grader.grade_2_3(make_bomb_circuit())


#### 이제 폭탄이 있는 회로를 돌리고 결과 해석을 해봅시다.

폭탄 탐지기 회로는 두 개의 클래시컬 비트(c1, c0)에 결과를 저장합니다.

- `c1`: `q1`의 측정 결과. `c1=1`이면 폭탄이 터졌음을 의미합니다.

- `c0`: `q0`의 측정 결과. `c0=0`이면, Detector 2에서 광자가 검출된거고, `c0=1`이면 폭탄을 터트리지 않고 성공적으로 탐지했음을 의미합니다(Detector 1 검출).

### **문제 2-4 :**

다음을 수행하는 코드를 완성하세요 :

1. `AerSimulator`를 백엔드로 해서 `Sampler`로 `bomb_circuit`을 `shots=1024`로 실행합니다.

2. `c0`와 `c1`의 결과에 따라 `interpreted_counts` 딕셔너리에 해당하는 count 수를 추가합니다.
    - `'detected w/o exploding'`: 폭탄을 터트리지 않고 성공적으로 감지한 경우
    - `'exploded'`: 폭탄이 터진 경우
    - `'NA'`: 아무일도 일어나지 않은 경우 

4. `interpreted_counts`의 히스토그램을 그립니다.

In [ ]:
bomb_circuit = # TODO
# TODO

interpreted_counts = {"detected w/o exploding": 0, "exploded": 0, "NA": 0}

# TODO

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
bomb_circuit.draw("mpl", ax=axes[0])
plot_histogram(interpreted_counts, ax=axes[1])
plt.show()

In [ ]:
# Grader Cell
grader.grade_2_4(interpreted_counts)

### 탐지 효율 점수

Mach-Zehnder 간섭계의 성능을 평가하기 위한 탐지 효율 점수가 있습니다.

이 점수는 폭탄을 덜 터트리고 성공적으로 잘 감지할 수록 높아집니다.

점수는 `grader.get_score(interpreted_counts)`로 구할 수 있습니다.

#####  ‼️ `interpreted_counts`의 key들을 절대 바꾸지 마세요 ‼️

아래 셀을 실행해 `H`로 beam splitter를 만들었을 때의 점수를 확인해보세요

In [ ]:
print("Score:", grader.get_score(interpreted_counts))

---

### 비율 조절 가능한 빔 스플리터로 성능 높이기

이전 문제에서는 H게이트로 50:50 비율로 고정된 빔 스플리터를 사용했습니다. 

이번에는 rotational gate를 사용하여, splitting ratio을 조절하여 Mach-Zehnder 간섭계의 성능을 높인 회로를 구현합니다.

rx 또는 ry 게이트는 $x,y$축을 중심으로 큐비트를 $\theta$ 각도만큼 회전시킵니다. 이 각도 $\theta$를 조절하면, 초기 상태 $∣0\rangle$이 $∣0\rangle$과 $∣1\rangle$의 중첩 상태로 나뉘는 비율을 제어할 수 있습니다. 

이때 두번째 빔 스플리터는 완벽한 상쇄 간섭을 일으켜, 아무런 방해가 없을 때 광자가 항상 예측 가능한 하나의 결과로만 나오도록 만들기 위해서 첫번째 빔 스플리터의 정확한 역연산이 되어야합니다.
(첫 번째 스플리터가 $U=AB$ 라면, 두 번째는 $U^\dagger=B^\dagger A^\dagger$ 어야 함)

### 🏆 챌린지: 최고의 양자 폭탄 탐지기를 설계하세요!

이전 단계에서 우리는 고정된 50:50 빔 스플리터(H 게이트)를 사용한 기본적인 폭탄 탐지기를 구현했습니다. 그 결과, 효율 점수(grader.get_score())가 약 60-70점 수준에 머물렀죠.

이번 최종 챌린지에서는 이 탐지기의 성능을 극한까지 끌어올려야 합니다! 빔 스플리터를 조절 가능한 회전 게이트(Rx 또는 Ry)로 교체하고, 최적의 회전 각도(θ)를 찾아 탐지 효율 점수(grader.get_score())를 극대화 하세요.

이 챌린지는 경쟁입니다! grader.get_score() 함수를 통해 얻은 최종 점수가 가장 높은 참가자에게는 경품이 지급될 예정입니다.

다양한 아이디어를 동원해 탐지 효율 점수를 최대한 높여보세요!

Rotational gate를 사용해 폭탄이 있는 간섭계 회로 `make_rot_circuit()`을 만들고, AerSimulator로 회로응 돌려서 `rot_interpreted_counts`를 얻어 효율 점수를 구하세요.

In [ ]:
# def make_rot_cirtuit():
#     # TODO

# TODO

rot_interpreted_counts = {"detected w/o exploding": 0, "exploded": 0, "NA": 0}

# TODO

print("Rotational Circuit Score:", grader.get_score(rot_interpreted_counts))

#### 최종 rotational circuit을 `final_rot_circuit` 변수에 저장해서 `grader.grade_challenge`에 넣어서 실행해주세요.

#### 이 점수가 제출됩니다.

In [ ]:
# Grader Cell
grader.grade_challenge(final_rot_circuit)

---
#### 아래 셀을 실행해 최종 제출 해주세요.


In [ ]:
grader.final_submission()

<br><br><br>

#### &copy; 2025 Quantum Informatics at Yonsei Academy. All Rights Reserved

<details>
<summary style="font-size: large; font-weight: bold;">Credits</summary>

Notebook by Justin J. Kim and Jeongbin Cho

Grader by Justin J. Kim and Jeongbin Cho

Contact : j.hwankim@yonsei.ac.kr, jeongbin033@yonsei.ac.kr
</details>